<a href="https://colab.research.google.com/github/shaaagri/iat481-nlp-proj/blob/main/LLama2_vanilla_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
In this preparation notebook, we will set up a suitable variant of Llama2 LLM to be able to do some raw queries. We then will try to integrate it into a langchain-based pipeline and test it by asking the model some simple questions (a "vanilla" chatbot, missing yet any customization). This will help us to get ready for the next iteration in which we will take this pipeline one step further by implementing RAG (Retrieval Augmented Generation).

We begin by building on top of "Chatbot_LLama_2.ipynb" notebook from our tutorials.

## Workflow
1. **Installations**: We'll begin by setting up our environment with the required libraries.
2. **Prerequisites**: Ensure we have access to the Llama-2 7B model on Hugging Face.
3. **Loading the Model & Tokenizer**: Retrieve the model and tokenizer for our session.
4. **Creating the Llama Pipeline**: Prepare our model for generating responses.
5. **Interacting with Llama**: Prompt the model for answers and explore its capabilities.

Let's dive in!
